In [1]:
# Imports nécessaires
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from io import BytesIO
from IPython.display import Audio, display
%matplotlib inline

def get_extension(source, upload_value, file_path):
    """Retourne l'extension du fichier si présent."""
    if source == 'Sinus':
        return None
    # Chemin local
    if isinstance(file_path, str) and file_path.strip():
        return os.path.splitext(file_path.strip())[1].lower()
    # Upload widget
    files = upload_value.value if hasattr(upload_value, 'value') else upload_value
    if files:
        if isinstance(files, dict):
            meta, _ = next(iter(files.items()))
            name = meta.get('name', '')
        else:
            info = files[0]
            name = info.get('name', '')
        return os.path.splitext(name)[1].lower()
    return None


In [2]:
def load_signal(source, upload_value, file_path):
    # Sinus
    if source == 'Sinus':
        t = np.linspace(0, 1, 1000, endpoint=False)
        return np.sin(2*np.pi*5*t), 1000
    # Fichier
    if source == 'Fichier':
        # Chemin local
        path = file_path.strip() if isinstance(file_path, str) else ''
        if path and os.path.isfile(path):
            ext = os.path.splitext(path)[1].lower()
            if ext == '.wav':
                fs, x = wavfile.read(path)
                if x.ndim == 2: x = x[:,0]
                return x.astype(np.float32), fs
            elif ext in ('.xlsx', '.xls'):
                df = pd.read_excel(path, header=None)
                return df.iloc[:,0].values, 1_000_000
        # Upload fallback
        files = upload_value.value if hasattr(upload_value, 'value') else upload_value
        if files:
            if isinstance(files, dict):
                meta, content = next(iter(files.items()))
                name = meta.get('name', '')
                bio = BytesIO(content)
            else:
                info = files[0]
                name = info.get('name', '')
                bio = BytesIO(info.get('content', b''))
            ext = os.path.splitext(name)[1].lower()
            if ext == '.wav':
                fs, x = wavfile.read(bio)
                if x.ndim == 2: x = x[:,0]
                return x.astype(np.float32), fs
            elif ext in ('.xlsx', '.xls'):
                df = pd.read_excel(bio, header=None)
                return df.iloc[:,0].values, 1_000_000
    # Par défaut
    t = np.linspace(0, 1, 1000, endpoint=False)
    return np.sin(2*np.pi*5*t), 1000

def process_signal(module, x, fs, facteur_ech, bits,
                   type_fenetre, type_filtre, f_basse, f_haute, ordre, zp_factor):
    sig = x.copy()
    fs_local = fs
    if module == 'Échantillonnage':
        N_new = max(1, int(len(sig) * facteur_ech))
        sig = signal.resample(sig, N_new)
        fs_local = int(fs * facteur_ech)
        if bits > 0:
            levels = 2**bits
            mn, mx = sig.min(), sig.max()
            norm = (sig - mn) / (mx - mn)
            sig = (np.round(norm*(levels-1))/(levels-1))*(mx-mn)+mn
    elif module == 'Fenêtrage':
        win = signal.get_window(type_fenetre, len(sig))
        sig = sig * win
    elif module == 'Filtrage':
        nyq = fs_local / 2
        lb = max(f_basse, 1e-6)
        hb = min(f_haute, nyq - 1e-6)
        if hb <= lb:
            hb = lb + 1e-6
        # Design filter taps
        if type_filtre == 'Passe-bas':
            taps = signal.firwin(ordre+1, hb/nyq, pass_zero=True)
        elif type_filtre == 'Passe-haut':
            taps = signal.firwin(ordre+1, lb/nyq, pass_zero=False)
        elif type_filtre == 'Passe-bande':
            taps = signal.firwin(ordre+1, [lb/nyq, hb/nyq], pass_zero=False)
        elif type_filtre == 'Coupe-bande':
            taps = signal.firwin(ordre+1, [lb/nyq, hb/nyq], pass_zero=True)
        else:
            raise ValueError(f"Type de filtre inconnu: {type_filtre}")
        sig = signal.lfilter(taps, 1.0, sig)
    elif module == 'Zero padding':
        # Zero padding pour FFT: ajouter (zp_factor-1)*len(sig) zéros
        pad_len = int((zp_factor - 1) * len(sig))
        sig = np.pad(sig, (0, pad_len), mode='constant')
    return sig, fs_local


In [3]:
def plot_comparison(module, source, upload_value, file_path,
                    facteur_ech, bits, type_fenetre,
                    type_filtre, f_basse, f_haute,
                    ordre, zp_factor, t_debut, t_fin,
                    f_debut, f_fin):
    x, fs = load_signal(source, upload_value, file_path)
    y, fs2 = process_signal(module, x, fs,
                             facteur_ech, bits,
                             type_fenetre, type_filtre,
                             f_basse, f_haute,
                             ordre, zp_factor)
    # Domaines temps et masques
    t1 = np.arange(len(x))/fs; t2 = np.arange(len(y))/fs2
    mask_t1 = (t1>=t_debut)&(t1<=t_fin); mask_t2 = (t2>=t_debut)&(t2<=t_fin)
    # Spectres et masques
    f1 = np.fft.rfftfreq(len(x), 1/fs); A1 = (2.0/len(x))*np.abs(np.fft.rfft(x))
    f2 = np.fft.rfftfreq(len(y), 1/fs2); A2 = (2.0/len(x))*np.abs(np.fft.rfft(y))
    mask_f1 = (f1>=f_debut)&(f1<=f_fin); mask_f2 = (f2>=f_debut)&(f2<=f_fin)

    fig, axs = plt.subplots(1,2, figsize=(12,4))
    # Domaine temps
    if module == 'Échantillonnage':
        axs[0].plot(t1[mask_t1], x[mask_t1], label='Original', color='C0', linestyle='-')
        axs[0].scatter(t2[mask_t2], y[mask_t2], label='Échantillonné', color='C1', s=10)
    else:
        axs[0].plot(t1[mask_t1], x[mask_t1], label='Original', color='C0', linestyle='-')
        axs[0].plot(t2[mask_t2], y[mask_t2], label=f'Modifié ({module})', color='C1', linestyle='--')
    axs[0].set_title('Domaine temps'); axs[0].set_xlabel('Temps (s)'); axs[0].set_ylabel('Amplitude')
    axs[0].legend(); axs[0].grid(True)
    # Domaine fréquence
    axs[1].plot(f1[mask_f1], A1[mask_f1], label='Original', color='C0', linestyle='-')
    axs[1].plot(f2[mask_f2], A2[mask_f2], label=f'Modifié ({module})', color='C1', linestyle='--')
    axs[1].set_title('Domaine fréquence'); axs[1].set_xlabel('Fréquence (Hz)'); axs[1].set_ylabel('Amplitude spectrale')
    axs[1].legend(); axs[1].grid(True)
    plt.tight_layout()
    plt.show()

    # Audio uniquement sur wav
    ext = get_extension(source, upload_value, file_path)
    if ext == '.wav':
        display(Audio(data=y, rate=fs2))


In [4]:
import ipywidgets as widgets
from ipywidgets import interactive_output

# Source
source_w = widgets.Dropdown(options=['Sinus','Fichier'], description='Source:')
file_path_w = widgets.Text(description='Chemin fichier:', placeholder='ex: /path/file.wav')
upload_w = widgets.FileUpload(description='Charger fichier')

# Module
module_w = widgets.Dropdown(options=['Échantillonnage','Fenêtrage','Filtrage','Zero padding'],
                            description='Module:')

# Paramètres
facteur_ech_w = widgets.FloatSlider(description='Taux x', min=0.01, max=1.0, step=0.01, value=1.0)
bits_w = widgets.IntSlider(description='Bits', min=0, max=16, value=8)
type_fenetre_w = widgets.Dropdown(options=['boxcar','hann','hamming','blackman'], description='Fenêtre:')
type_filtre_w = widgets.Dropdown(options=['Passe-bas','Passe-haut','Passe-bande','Coupe-bande'], description='Filtre:')
f_basse_w = widgets.FloatSlider(description='F basse', min=0, max=200000, step=100, value=1000)
f_haute_w = widgets.FloatSlider(description='F haute', min=0, max=200000, step=100, value=5000)
ordre_w = widgets.IntSlider(description='Ordre', min=1, max=64, value=8)
zp_factor_w = widgets.FloatSlider(description='ZP fact', min=1, max=64, step=0.5, value=1)

# Sliders temps/fréquence visibles
t_debut_w = widgets.FloatSlider(description='t0 (s)', min=0, max=1, step=0.01, value=0)
t_fin_w   = widgets.FloatSlider(description='t1 (s)', min=0, max=1, step=0.01, value=1)
f_debut_w = widgets.FloatSlider(description='f0 (Hz)', min=0, max=200000, step=50, value=0)
f_fin_w   = widgets.FloatSlider(description='f1 (Hz)', min=0, max=200000, step=50, value=200000)

# Grouper et cacher
param_widgets = [facteur_ech_w, bits_w, type_fenetre_w, type_filtre_w,
                 f_basse_w, f_haute_w, ordre_w, zp_factor_w]
for w in param_widgets:
    w.layout.display = 'none'

def on_module_change(change):
    for w in param_widgets:
        w.layout.display = 'none'
    if change['new']=='Échantillonnage':
        for w in [facteur_ech_w, bits_w]:
            w.layout.display = ''
    elif change['new']=='Fenêtrage':
        type_fenetre_w.layout.display = ''
    elif change['new']=='Filtrage':
        for w in [type_filtre_w, f_basse_w, f_haute_w, ordre_w]:
            w.layout.display = ''
    elif change['new']=='Zero padding':
        zp_factor_w.layout.display = ''

module_w.observe(on_module_change, names='value')
on_module_change({'new': module_w.value})

# Interaction
out = interactive_output(
    plot_comparison,
    {
        'module': module_w,
        'source': source_w,
        'upload_value': upload_w,
        'file_path': file_path_w,
        'facteur_ech': facteur_ech_w,
        'bits': bits_w,
        'type_fenetre': type_fenetre_w,
        'type_filtre': type_filtre_w,
        'f_basse': f_basse_w,
        'f_haute': f_haute_w,
        'ordre': ordre_w,
        'zp_factor': zp_factor_w,
        't_debut': t_debut_w,
        't_fin': t_fin_w,
        'f_debut': f_debut_w,
        'f_fin': f_fin_w
    }
)


# Ajustement dynamique des sliders de temps selon la durée du signal
def update_time_sliders(*args):
    x, fs = load_signal(source_w.value, upload_w.value, file_path_w.value)
    duration = len(x) / fs
    step = duration / 100
    # Nombre de décimales basé sur la durée du signal
    s = f"{duration:.10f}".rstrip('0')
    if '.' in s:
        decimals = len(s.split('.')[1])
    else:
        decimals = 1
    fmt = f'.{decimals}f'
    for w in (t_debut_w, t_fin_w):
        w.max = duration
        w.step = step
        w.readout_format = fmt
        # Ajuster la valeur si hors bornes
        if w.value > duration:
            w.value = duration if w is t_fin_w else 0

# Observers pour recalculer lors du changement de source/fichier
for widget in (source_w, upload_w, file_path_w):
    widget.observe(update_time_sliders, names='value')

# Initialisation
update_time_sliders()

display(widgets.VBox([
    widgets.HBox([source_w, file_path_w]),
    upload_w,
    module_w,
    widgets.HBox([facteur_ech_w, bits_w]),
    type_fenetre_w,
    widgets.HBox([type_filtre_w, f_basse_w, f_haute_w, ordre_w]),
    zp_factor_w,
    widgets.HBox([t_debut_w, t_fin_w]),
    widgets.HBox([f_debut_w, f_fin_w]),
    out
]))